In [1]:
import matplotlib.pyplot as plt
import numpy as np
import seaborn as sns
import subprocess

In [2]:
def convert_file_mymedialite(file_top_n, output_file):
    # read the training data and mount the matrix
    file_top_n = open(file_top_n, 'r')

    file_out = open(output_file, 'w')

    lines = []
    ids = []
    ratings = []
    dic_u_top = {}
    cont_u = 0
    for line in file_top_n:
        aux = line.split()
        user = aux[0]
        aux = aux[1][1:]
        aux = aux[:-1].split(",")
        file_out.write("%s %s\n" % (user, " ".join(aux)))

## Plot

In [ ]:
def plot_graphs(listX, listY, labelX, labelY, out='nothing', t='nothing'):
    fig, ax = plt.subplots()
    
    plt.plot(listX, listY, linewidth=3.0)

    ax.set_xlabel(labelX, fontsize='xx-large', labelpad=25, weight='semibold')
    ax.set_ylabel(labelY, fontsize='xx-large', labelpad=25, weight='semibold')
    
    ax.set_ylim(0, 1)

    plt.tick_params(axis='both', labelsize=20, pad=25)

    # for tick in ax.xaxis.get_ticklabels():
    #    tick.set_fontsize('x-large')
    #    tick.set_weight('bold')

    # for tick in ax.yaxis.get_ticklabels():
    #    tick.set_fontsize('x-large')
    #    tick.set_weight('bold')

    plt.tight_layout()

    if t != 'nothing':
        plt.title(t, fontsize='xx-large', weight='semibold')

    #plt.savefig(out)
    plt.show()

### Load Test Dataset ML1M

In [ ]:
filein = open('Datasets/ML-1M/ratings_test.txt', 'r')

u_rating_test = {}
for line in filein:
    line = line.strip()
    try:
        values = line.split("::")
        u_id = values[0]
        i_id = values[1]
        r = values[2]
        
        if not u_rating_test.get(u_id, False):
            u_rating_test[u_id] = {}
        u_rating_test[u_id][i_id] = r
        
    except:
        print(line)

### Load Recommender List and Evaluation

In [ ]:
rec_list = ['rec_itemKNN', 'rec_MostPopular', 'rec_userKNN', 'rec_WRMF']
rec_n = ['', '_5', '_10', '_20']

In [ ]:
rec_list = ['rec_WRMF']
rec_n = ['_20']

In [ ]:
for r_list in rec_list:
    for r_n in rec_n:
        filein = open("Recommendations-Lists/" +r_list+ "" +r_n+ ".txt", 'r')

        u_rating_pred = {}
        for line in filein:
            line = line.strip()
            try:
                values = line.split("\t")
                u_id = values[0]
                values = values[1][1:-1].split(",")

                for i in values:
                    v = i.split(":")
                    i_id = v[0]
                    r = v[1]

                    if not u_rating_pred.get(u_id, False):
                        u_rating_pred[u_id] = {}
                    u_rating_pred[u_id][i_id] = r

            except:
                print(line)

        ################
        ### ACCURACY ###
        ################

        print(n)
        print(r_list)
        print(r_n)
        fileout = open("Evaluations/Standard/" +r_list+ "/" +r_list+ "" +r_n+ "_acc.txt", 'w')
        fileout2 = open("Evaluations/Standard/" +r_list+ "/" +r_list+ "_eval.txt", 'a')

        u_acc = {}
        acc_median = []
        acc_average = []
        for i,user in enumerate(u_rating_pred):
            u_acc[user] = 0
            for j, item in enumerate(u_rating_pred[user]):
                if u_rating_test[user].get(item, False):
                    u_acc[user] += 1

            u_acc[user] /= len(u_rating_pred[user].keys())
            acc_average.append(u_acc[user])
            acc_median.append(u_acc[user])

        fileout2.write("\n")
        fileout2.write("%s%s\n" % (r_list,r_n))
        fileout2.write("Accuracy Average: %f\n" % (np.mean(acc_average)))
        fileout2.write("Accuracy Median: %f\n" % (np.median(acc_median)))
        fileout2.write("-------------------------------\n")
        for i, v in enumerate(u_acc):
            fileout.write("%s\t%f\n" % (v, u_acc[v]))

        #plot
        #listY = list(u_acc.values())
        #listY.sort(reverse=True)
        #print(listY[0])
        #plot_graphs((np.arange(len(u_acc.keys())) / len(u_acc.keys())) * 100, listY, "Users", "Acc", out='nothing', t=rec_list)


## Diversity

In [ ]:
convert_file_mymedialite("Recommendations-Lists/rec_itemKNN.txt", "Recommendations-Lists/rec_itemKNN_conv.txt")

In [ ]:
command = "cd frameworks/vargas/src/ && make && ./getMetrics " \
              "-b ../../../Datasets/ML-1M/ratings_train.txt " \
              "-p ../../../Recommendations-Lists/" +rec_list+ "_conv.txt " \
              "-o ../../../Evaluations/Standard/" +rec_list+ "_div.txt " \
              "-l ../../../Datasets/ML-1M/ratings_test.txt "\
              "-n 100"\
              " && make clean"
print(command)
subprocess.call(command, shell=True)

In [3]:
#rec_list = ['rec_itemKNN_5', 'rec_itemKNN_10', 'rec_itemKNN_20',
#                 'rec_MostPopular', 'rec_MostPopular_5', 'rec_MostPopular_10', 'rec_MostPopular_20',
#                 'rec_userKNN', 'rec_userKNN_5', 'rec_userKNN_10', 'rec_userKNN_20',
#                 'rec_WRMF', 'rec_WRMF_5', 'rec_WRMF_10', 'rec_WRMF_20']
#rec_list = ['rec_MostPopular', 'rec_MostPopular_5', 'rec_MostPopular_10', 'rec_MostPopular_20']
#rec_n = [5, 10, 20, 100, 5, 10, 20, 100, 5, 10, 20, 100, 5, 10, 20]
rec_list = ['rec_MostPopular']
rec_n = [5]
for i in range(len(rec_list)):
    convert_file_mymedialite("Recommendations-Lists/"+rec_list[i]+".txt", "Recommendations-Lists/"+rec_list[i]+"_conv.txt")
    command = "cd frameworks/vargas/src/ && make && ./getMetrics " \
              "-b ../../../Datasets/ML-1M/ratings_train.txt " \
              "-p ../../../Recommendations-Lists/" +rec_list[i]+ "_conv.txt " \
              "-o ../../../Evaluations/Standard/" +rec_list[i]+ "_div.txt " \
              "-l ../../../Datasets/ML-1M/ratings_test.txt "\
              "-n "+ str(rec_n[i]) +" "\
              "-t 4"\
              " && make clean"
    print(command)
    subprocess.call(command, shell=True)

cd frameworks/vargas/src/ && make && ./getMetrics -b ../../../Datasets/ML-1M/ratings_train.txt -p ../../../Recommendations-Lists/rec_MostPopular_conv.txt -o ../../../Evaluations/Standard/rec_MostPopular_div.txt -l ../../../Datasets/ML-1M/ratings_test.txt -n 5 -t 4 && make clean
